In [1]:
from __future__ import annotations

class Fruit:
    def __init__(self, name, color) -> None:
        self.name = name
        self.color = color

class Apple:
    def __init__(self, color, weight, price, origin):
        self.color = color
        self.weight = weight
        self.price = price
        self.origin = origin
    
    def get_fruit(self) -> Fruit:
        return Fruit("apple", self.color)
    


In [2]:
from typing import Any

def get_type_annotation(data: Any) -> str:
    # Check if the input is a dictionary
    if isinstance(data, dict):
        # Get the class name of the first non-None value in the dictionary
        class_name = next((value.__class__.__name__ for value in data.values() if value is not None), None)

        if class_name:
            return f"dict[str, {class_name}]"
        else:
            return "dict[str, Any]"
    # If the input is not a dictionary, return its type
    else:
        return type(data).__name__

In [3]:
def get_object_string(obj):
    if isinstance(obj, str):
        return f'"{obj}"'
    elif isinstance(obj, (int, float, bool)):
        return str(obj)
    elif isinstance(obj, list):
        return '[' + ', '.join(get_object_string(item) for item in obj) + ']'
    elif isinstance(obj, tuple):
        return '(' + ', '.join(get_object_string(item) for item in obj) + ')'
    elif isinstance(obj, dict):
        return '{' + ', '.join(f"{get_object_string(key)}: {get_object_string(value)}" for key, value in obj.items()) + '}'
    elif hasattr(obj, '__dict__'):
        args = ', '.join(f"{key}={get_object_string(value)}" for key, value in vars(obj).items())
        return f"{obj.__class__.__name__}({args})"
    else:
        return str(obj)

In [4]:
apple1 = Apple("red", 100, 100, "USA")
a = {
    "a1": apple1,
    "a2": Apple("green", 200, 200, "Japan"),
    "a3": Apple("yellow", 300, 300, "China"),
}

In [5]:
get_type_annotation(a)

'dict[str, Apple]'

In [6]:
get_object_string(a)

'{"a1": Apple(color="red", weight=100, price=100, origin="USA"), "a2": Apple(color="green", weight=200, price=200, origin="Japan"), "a3": Apple(color="yellow", weight=300, price=300, origin="China")}'

In [7]:
a = Apple("red", 100, 10, "China")

In [8]:
get_type_annotation(a)

'Apple'

In [9]:
get_object_string(a)

'Apple(color="red", weight=100, price=10, origin="China")'

In [10]:
a = 10
print(get_type_annotation(a), get_object_string(a))

int 10


In [11]:
print(f"'Apple Examples' (a) ({get_type_annotation(a)}) = {get_object_string(a)}")

'Apple Examples' (a) (int) = 10


In [17]:
def get_var_name(var):
    """
    Returns the name of a variable.

    Args:
        var (any): The variable whose name needs to be retrieved.

    Returns:
        str: The name of the variable.
    """
    # Get the global and local variable names
    global_names = list(globals().keys())
    local_names = list(locals().keys())

    # Check if the variable is in the global scope
    for name in global_names:
        if id(globals()[name]) == id(var):
            return name

    # Check if the variable is in the local scope
    for name in local_names:
        if id(locals()[name]) == id(var):
            return name

    # If the variable was not found, return None
    return None

In [18]:

a = "hello"
b = 10
apple = Apple("red", 100, 10, "China")

get_var_name(a) # a
get_var_name(b) # b
get_var_name(apple) # apple

'apple'

In [21]:
print(get_var_name(b))

b


In [20]:
get_var_name(apple)

'apple'

In [94]:
registry_data = {
    "get_emoji(Module)": {
        "model": ["obj", ""],
        "llm": [None, ""],
        "emoji_examples": ["list[dict[str,str]]", "Examples of Text to Emoji"],
        "PersonalityIndex": ["class", "Personality Index of a Person"],
        "Personality": ["Enum", "Personality of the Person"],
        "personality_examples": [
            "dict[str,Personality|None]",
            "Personality Information of Famous People",
        ],
        "Person": ["obj", "Person"],
        "outer": ["obj", "main object "],
        "obj1": [None, ""],
        "pp": [None, ""],
    },
    "get_emoji(Module).PersonalityIndex(class)": {"": [None, ""]},
    "get_emoji(Module).Personality(Enum)": {
        "INTROVERT": [None, "Person who is shy and reticent"],
        "EXTROVERT": [None, "Person who is outgoing and socially confident"],
    },
    "get_emoji(Module).Person(obj)": {
        "name": ["str", "Name of the Person"],
        "age": ["int", "Age of the Person"],
        "personality": ["Personality", "Personality of the Person"]
    },
    "get_emoji(Module).outer(obj).inner(obj)": {"in_var": ["int|Person", "inner variable"]},
    "get_emoji(Module).outer(obj)": {"inner": ["obj", "inner object"]},
}

In [117]:
import re

def get_type_explanation(type_str: str) -> str:
    main_registry_type_info = None
    scope = None
    for k, v in registry_data.items():
        for i, j in v.items():
            if i == type_str:
                main_registry_type_info = j
                scope = k
                break
    if not main_registry_type_info:
        return
    type_type = main_registry_type_info[0]
    type_semstr = main_registry_type_info[1]
    type_info = registry_data[f"{scope}.{type_str}({type_type})"]
    type_info_str = []
    type_info_types = []
    if type_type == "Enum":
        for k, v in type_info.items():
            type_info_str.append(f"{v[1]} ({k}) (EnumItem)")
    elif type_type in ["obj", "class", "node", "edge"]:
        for k, v in type_info.items():
            type_info_str.append(f"{v[1]} ({k}) ({v[0]})")
            if extract_non_primary_type(v[0]):
                type_info_types.extend(extract_non_primary_type(v[0]))
    return f"{type_semstr} ({type_str}) ({type_type}) = {', '.join(type_info_str)}", set(type_info_types)


def extract_non_primary_type(type_str: str) -> list:
    """Extract non-primary types from the type string."""
    if not type_str:
        return []
    pattern = r"(?:\[|,\s*|\|)([a-zA-Z_][a-zA-Z0-9_]*)|([a-zA-Z_][a-zA-Z0-9_]*)"
    matches = re.findall(pattern, type_str)
    primary_types = [
        "str",
        "int",
        "float",
        "bool",
        "list",
        "dict",
        "tuple",
        "set",
        "Any",
        "None",
    ]
    non_primary_types = [m for t in matches for m in t if m and m not in primary_types]
    return non_primary_types

def get_all_type_explanations(type_list: list) -> str:
    collected_type_explanations = {}
    for type_item in type_list:
        type_explanation, nested_types = get_type_explanation(type_item)
        if type_item not in collected_type_explanations:
            collected_type_explanations[type_item] = type_explanation
            if nested_types:
                nested_collected_type_explanations = get_all_type_explanations(nested_types)
                for k, v in nested_collected_type_explanations.items():
                    if k not in collected_type_explanations:
                        collected_type_explanations[k] = v
    return collected_type_explanations

In [118]:
get_type_explanation("Personality")

('Personality of the Person (Personality) (Enum) = Person who is shy and reticent (INTROVERT) (EnumItem), Person who is outgoing and socially confident (EXTROVERT) (EnumItem)',
 set())

In [119]:
get_type_explanation("Person")

('Person (Person) (obj) = Name of the Person (name) (str), Age of the Person (age) (int), Personality of the Person (personality) (Personality)',
 {'Personality'})

In [120]:
get_type_explanation("PersonalityIndex")

('Personality Index of a Person (PersonalityIndex) (class) =  () (None)',
 set())

In [121]:
get_type_explanation("inner")

('inner object (inner) (obj) = inner variable (in_var) (int|Person)',
 {'Person'})

In [122]:
print("\n".join(list(get_all_type_explanations([ "Person", "Personality" ,"PersonalityIndex", "inner"]).values())))

['Person', 'Personality', 'PersonalityIndex', 'inner'] Person {'Personality'}
{'Personality'} Personality set()
['Person', 'Personality', 'PersonalityIndex', 'inner'] Personality set()
['Person', 'Personality', 'PersonalityIndex', 'inner'] PersonalityIndex set()
['Person', 'Personality', 'PersonalityIndex', 'inner'] inner {'Person'}
{'Person'} Person {'Personality'}
{'Personality'} Personality set()
Person (Person) (obj) = Name of the Person (name) (str), Age of the Person (age) (int), Personality of the Person (personality) (Personality)
Personality of the Person (Personality) (Enum) = Person who is shy and reticent (INTROVERT) (EnumItem), Person who is outgoing and socially confident (EXTROVERT) (EnumItem)
Personality Index of a Person (PersonalityIndex) (class) =  () (None)
inner object (inner) (obj) = inner variable (in_var) (int|Person)


In [134]:
from typing import Any


def get_type_annotation(data: Any) -> str:
    # Check if the input is a dictionary
    if isinstance(data, dict):
        # Get the class name of the first non-None value in the dictionary
        class_name = next(
            (value.__class__.__name__ for value in data.values() if value is not None),
            None,
        )

        if class_name:
            return f"dict[str, {class_name}]"
        else:
            return "dict[str, Any]"
    # If the input is not a dictionary, return its type
    else:
        return type(data).__name__


def get_object_string(obj):
    if isinstance(obj, str):
        return f'"{obj}"'
    elif isinstance(obj, (int, float, bool)):
        return str(obj)
    elif isinstance(obj, list):
        return '[' + ', '.join(get_object_string(item) for item in obj) + ']'
    elif isinstance(obj, tuple):
        return '(' + ', '.join(get_object_string(item) for item in obj) + ')'
    elif isinstance(obj, dict):
        return '{' + ', '.join(f"{get_object_string(key)}: {get_object_string(value)}" for key, value in obj.items()) + '}'
    elif hasattr(obj, '__dict__'):
        args = ', '.join(f"{key}={get_object_string(value)}" for key, value in vars(obj).items())
        return f"{obj.__class__.__name__}({args})"
    else:
        return str(obj)


def get_variable_name(var):
    """
    Returns the name of a variable.

    Args:
        var (any): The variable whose name needs to be retrieved.

    Returns:
        str: The name of the variable.
    """
    # Get the global and local variable names
    global_names = list(globals().keys())
    local_names = list(locals().keys())

    # Check if the variable is in the global scope
    for name in global_names:
        if id(globals()[name]) == id(var):
            return name

    # Check if the variable is in the local scope
    for name in local_names:
        if id(locals()[name]) == id(var):
            return name

    # If the variable was not found, return None
    return None

In [135]:
a = "hello"

from enum import Enum

Personality = Enum("Personality", ["INTROVERT", "EXTROVERT"])

In [136]:
Personality

<enum 'Personality'>

In [149]:
get_type_annotation(a)

'str'

In [161]:
class Person:
    def __init__(self, name, age, personality):
        self.name = name
        self.age = age
        self.personality = personality

In [162]:
get_type_annotation(Person)

'type'

In [163]:
x = {
    "a1": Person("John", 25, Personality.INTROVERT),
    "a2": Person("Jane", 30, Personality.EXTROVERT),
    "a3": Person("Doe", 35, Personality.INTROVERT)
}

In [164]:
get_object_string(x)

'{"a1": Person(name="John", age=25, personality=Personality.INTROVERT), "a2": Person(name="Jane", age=30, personality=Personality.EXTROVERT), "a3": Person(name="Doe", age=35, personality=Personality.INTROVERT)}'

In [165]:
from enum import Enum

def get_object_string(obj):
    if isinstance(obj, str):
        return f'"{obj}"'
    elif isinstance(obj, (int, float, bool)):
        return str(obj)
    elif isinstance(obj, list):
        return '[' + ', '.join(get_object_string(item) for item in obj) + ']'
    elif isinstance(obj, tuple):
        return '(' + ', '.join(get_object_string(item) for item in obj) + ')'
    elif isinstance(obj, dict):
        return '{' + ', '.join(f"{get_object_string(key)}: {get_object_string(value)}" for key, value in obj.items()) + '}'
    elif isinstance(obj, Enum):
        return f"{obj.__class__.__name__}.{obj.name}"
    elif hasattr(obj, '__dict__'):
        args = ', '.join(f"{key}={get_object_string(value)}" for key, value in vars(obj).items())
        return f"{obj.__class__.__name__}({args})"
    else:
        return str(obj)

a = 10
b ="apple"
c = [1,2,3]
p = Person("John", 25, Personality.INTROVERT)


x = {
    "a1": Person("John", 25, Personality.INTROVERT),
    "a2": Person("Jane", 30, Personality.EXTROVERT),
    "a3": Person("Doe", 35, Personality.INTROVERT)
}

print(get_object_string(a)) # 10
print(get_object_string(b)) # apple
print(get_object_string(c)) # [1,2,3]
print(get_object_string(p)) # Person(name="John", age=25, personality=Personality.INTROVERT)
print(get_object_string(x)) # {'a1': Person(name="John", age=25, personality=Personality.INTROVERT), 'a2': Person(name="Jane", age=30, personality=Personality.EXTROVERT), 'a3': Person(name="Doe", age=35, personality=Personality.INTROVERT)}



10
"apple"
[1, 2, 3]
Person(name="John", age=25, personality=Personality.INTROVERT)
{"a1": Person(name="John", age=25, personality=Personality.INTROVERT), "a2": Person(name="Jane", age=30, personality=Personality.EXTROVERT), "a3": Person(name="Doe", age=35, personality=Personality.INTROVERT)}


In [150]:
def get_type(obj):
    """
    Returns the type of the given object.
    """
    return type(obj).__name__

class Person:
    def __init__(self, name, age, personality):
        self.name = name
        self.age = age
        self.personality = personality

from enum import Enum

Personality = Enum("Personality", ["INTROVERT", "EXTROVERT"])

get_type(Person) # Person
get_type(Personality) # Personality


'EnumType'

In [167]:
from jaclang.plugin.feature import JacFeature as _Jac
from dataclasses import dataclass as __jac_dataclass__

@_Jac.make_obj(on_entry=[], on_exit=[])
@__jac_dataclass__(eq=False)
class Person2:
    name: str
    age: int
    personality: Personality

In [181]:
def get_object_string(obj):
    if isinstance(obj, str):
        return f'"{obj}"'
    elif isinstance(obj, (int, float, bool)):
        return str(obj)
    elif isinstance(obj, list):
        return '[' + ', '.join(get_object_string(item) for item in obj) + ']'
    elif isinstance(obj, tuple):
        return '(' + ', '.join(get_object_string(item) for item in obj) + ')'
    elif isinstance(obj, dict):
        return '{' + ', '.join(f"{get_object_string(key)}: {get_object_string(value)}" for key, value in obj.items()) + '}'
    elif isinstance(obj, Enum):
        print(obj)
        return f"{obj.__class__.__name__}.{obj.name}"
    elif hasattr(obj, '__dict__'):
        args = ', '.join(f"{key}={get_object_string(value)}" for key, value in vars(obj).items() if key!="_jac_")
        return f"{obj.__class__.__name__}({args})"
    else:
        return str(obj)

In [182]:
a = Person2("John", 25, Personality.INTROVERT)

In [183]:
get_object_string(a) # Person2(name="John", age=25, personality=Personality.INTROVERT)

Personality.INTROVERT


'Person2(name="John", age=25, personality=Personality.INTROVERT)'

In [186]:
x = {
    "a1": Personality.INTROVERT,
    "a2": Personality.EXTROVERT,
}

In [187]:
get_object_string(x)

Personality.INTROVERT
Personality.EXTROVERT


'{"a1": Personality.INTROVERT, "a2": Personality.EXTROVERT}'

In [188]:
obj_str = "Personality.INTROVERT"

obj = eval(obj_str)

In [189]:
obj

<Personality.INTROVERT: 1>

In [190]:
obj_str = "Person(name='John', age=25, personality=Personality.INTROVERT)"
obj = eval(obj_str)

In [191]:
obj

In [192]:
obj.personality

<Personality.INTROVERT: 1>

In [194]:

class Country:
    def __init__(self, name: str, code: Apple):
        self.name = name
        self.code = code
class Apple:
    def __init__(self, color: str, origin: Country = None):
        self.color = color
        self.origin = origin


In [211]:
obj_str = 'Apple(color="red", origin=Country(name="USA", code=Apple(color="red", origin=None)))'
obj = eval(obj_str)

In [213]:
obj.origin.code

In [18]:

import re
def get_reasoning_output(text):
    reasoning_match = re.search(r"\[Reasoning\]\s*(.*?)\s*\n", text)
    output_match = re.search(r"\[Output\]\s*(.*)", text)
    
    reasoning = reasoning_match.group(1) if reasoning_match else None
    output = output_match.group(1) if output_match else None
    
    return [reasoning, output]

In [46]:
text = '[Reasoning] J. Robert Oppenheimer, also known as the "father of the atomic bomb," was a brilliant theoretical physicist and the director of the Manhattan Project during World War II. He played a crucial role in developing the first nuclear weapons. However, after witnessing the devastation caused by the atomic bombs dropped on Hiroshima and Nagasaki, he expressed deep regret and became an advocate for nuclear disarmament. While he was an exceptional scientist, he was also known for his introspective and philosophical nature, which suggests an introverted personality.\\n[Output] Person(full_name="J. Robert Oppenheimer", yod=1967, personality=Personality.INTROVERT)'


In [47]:
get_reasoning_output(text)

['J. Robert Oppenheimer, also known as the "father of the atomic bomb," was a brilliant theoretical physicist and the director of the Manhattan Project during World War II. He played a crucial role in developing the first nuclear weapons. However, after witnessing the devastation caused by the atomic bombs dropped on Hiroshima and Nagasaki, he expressed deep regret and became an advocate for nuclear disarmament. While he was an exceptional scientist, he was also known for his introspective and philosophical nature, which suggests an introverted personality.\\n',
 'Person(full_name="J. Robert Oppenheimer", yod=1967, personality=Personality.INTROVERT)']

In [48]:
text1 = "[Reasoning] Something Reason \\n [Output] Something Output"
text2 = "[Output] Something"

In [49]:
get_reasoning_output(text1)

['Something Reason \\n', 'Something Output']

In [50]:
re.findall(r"\[Reasoning\](.*?)\\n\s*\[Output\](.*)", text)

[(' J. Robert Oppenheimer, also known as the "father of the atomic bomb," was a brilliant theoretical physicist and the director of the Manhattan Project during World War II. He played a crucial role in developing the first nuclear weapons. However, after witnessing the devastation caused by the atomic bombs dropped on Hiroshima and Nagasaki, he expressed deep regret and became an advocate for nuclear disarmament. While he was an exceptional scientist, he was also known for his introspective and philosophical nature, which suggests an introverted personality.',
  ' Person(full_name="J. Robert Oppenheimer", yod=1967, personality=Personality.INTROVERT)')]

In [51]:
reasoning_match = re.search(r'\[Reasoning\](.*?)
                        ', text)
output_match = re.search(r'\[Output\](.*)', text)

SyntaxError: unterminated string literal (detected at line 1) (3121925053.py, line 1)

In [52]:
import re

def get_reasoning_output(s):
    reasoning_match = re.search(r'\[Reasoning\](.*)\[Output\]', s)
    output_match = re.search(r'\[Output\](.*)', s)
    
    if reasoning_match and output_match:
        reasoning = reasoning_match.group(1)
        output = output_match.group(1)
        return [reasoning.strip(), output.strip()]
    elif output_match:
        output = output_match.group(1)
        return [None, output.strip()]
    else:
        return None

In [53]:
get_reasoning_output(text)

['J. Robert Oppenheimer, also known as the "father of the atomic bomb," was a brilliant theoretical physicist and the director of the Manhattan Project during World War II. He played a crucial role in developing the first nuclear weapons. However, after witnessing the devastation caused by the atomic bombs dropped on Hiroshima and Nagasaki, he expressed deep regret and became an advocate for nuclear disarmament. While he was an exceptional scientist, he was also known for his introspective and philosophical nature, which suggests an introverted personality.\\n',
 'Person(full_name="J. Robert Oppenheimer", yod=1967, personality=Personality.INTROVERT)']

In [45]:
get_reasoning_output(text1)

['Something Reason \\n', 'Something Output']